> Import packages

> Read in and inspect scraped data

In [1]:
%matplotlib inline
import pickle
from __future__ import division
import pandas as pd
import numpy as np

In [2]:
with open("my_data_2000_2014_02.pkl", 'r') as picklefile: 
    raw_data = pickle.load(picklefile)

In [3]:
data = pd.DataFrame(raw_data)
data = pd.DataFrame.transpose(data)

In [4]:
len(data)

4418

In [5]:
data.head()

,country,director,duration,genres,gross,language,lead,metacritic,num_reviews,num_users,opening,rating,release,studio,title,user_score
/title/tt0035423/,USA,James Mangold,PT118M,"[Comedy, Fantasy, Romance]","$47,095,453",English,Meg Ryan,44/100,311,"59,888","$2,562,448 \n\n (USA)",PG-13,2001-12-25,Konrad Pictures,Kate & Leopold,6.3
/title/tt0116282/,NaN,NaN,PT98M,"[Crime, Drama, Thriller]","$24,611,975",NaN,NaN,85/100,NaN,NaN,"$137,301 \n\n (USA)",R,1996-04-05,PolyGram Filmed Entertainment,Fargo,NaN
/title/tt0118589/,USA,Vondie Curtis-Hall,PT104M,"[Drama, Music, Romance]","$4,273,372",English,Mariah Carey,14/100,306,"18,856","$2,414,596 \n\n (USA)",PG-13,2001-09-21,Twentieth Century Fox Film Corporation,Glitter,2.1
/title/tt0118694/,Hong Kong,Kar Wai Wong,PT98M,"[Drama, Romance]","$2,734,044",Cantonese,Tony Chiu Wai Leung,85/100,345,"67,141","£92,227 \n\n (UK)",PG,2001-03-09,Block 2 Pictures,In the Mood for Love,8.1
/title/tt0118926/,Spain,John Malkovich,PT132M,"[Drama, Thriller, Crime]","$2,377,348",English,Javier Bardem,64/100,71,"5,355","£61,622 \n\n (UK)",R,2003-05-23,Fox Searchlight Pictures,The Dancer Upstairs,7


> Filter only rows with valid 'metacritic' field

In [6]:
data = data[data['metacritic'] != 'Metacritic Reviews']

In [7]:
len(data)

4099

> Clean up variables with extraneous characters.

> Filter out rows with gross not in dollars.

In [8]:
data['duration'] = data['duration'].map(lambda x: x.encode('ascii'))
data['duration'] = data['duration'].map(lambda x: int(x.replace('PT', 
                                        '').replace('M', '')))

In [9]:
dols = 0
pds = 0
oth = 0

for gross in data['gross']:
    if gross[0] == u'$':
        dols += 1
    elif gross[0] == u'£':
        pds += 1
    else:
        oth += 1
        
print "dols: %d, pds: %d, oth: %d" % (dols, pds, oth)

# gross - dols: 4096, pds: 1, oth: 1
# opening - dols: 3651, pds: 444, oth: 3

dols: 4096, pds: 1, oth: 2


In [10]:
data = data[data['gross'].str.contains(u'\$')]

In [11]:
len(data)

4096

In [12]:
data['gross'] = data['gross'].map(lambda x: int(x.replace('$', '').replace(',', '')))

In [13]:
data['opening'] = data['opening'].map(lambda x: x.split()[0])

In [14]:
data = data[data['opening'].str.contains("[u\$\£]")]

In [15]:
len(data)

4093

> Rough conversion of pounds to dollars for opening gross variable

In [16]:
opening = data['opening']

print opening[:10]
print type(opening)

def convert_pounds(lst):
    newlst = []
    for item in lst:
        item = item.encode('ascii', 'ignore')
        if item[0] == u'$':
            item = int(item.replace('$', '').replace(',', ''))
            newlst.append(item)
        else:
            item = int(item.replace('£', '').replace(',', '')) * 1.5
            newlst.append(item)
    return newlst

opening2 = convert_pounds(opening)
print opening2[:10]

data['opening'] = opening2

/title/tt0035423/     $2,562,448
/title/tt0116282/       $137,301
/title/tt0118589/     $2,414,596
/title/tt0118694/        £92,227
/title/tt0118926/        £61,622
/title/tt0120202/       $580,163
/title/tt0120630/    $17,506,162
/title/tt0120667/     $2,815,167
/title/tt0120679/       $205,996
/title/tt0120681/    $11,014,818
Name: opening, dtype: object
<class 'pandas.core.series.Series'>
[2562448, 137301, 2414596, 138340.5, 92433.0, 580163, 17506162, 2815167, 205996, 11014818]


In [17]:
data.dtypes

country         object
director        object
duration         int64
genres          object
gross            int64
language        object
lead            object
metacritic      object
num_reviews     object
num_users       object
opening        float64
rating          object
release         object
studio          object
title           object
user_score      object
dtype: object

> Convert user score to float.

In [18]:
data['user_score'] = data['user_score'].map(lambda x: float(str(x)))

In [19]:
data.head()

,country,director,duration,genres,gross,language,lead,metacritic,num_reviews,num_users,opening,rating,release,studio,title,user_score
/title/tt0035423/,USA,James Mangold,118,"[Comedy, Fantasy, Romance]",47095453,English,Meg Ryan,44/100,311,"59,888",2562448.0,PG-13,2001-12-25,Konrad Pictures,Kate & Leopold,6.3
/title/tt0116282/,NaN,NaN,98,"[Crime, Drama, Thriller]",24611975,NaN,NaN,85/100,NaN,NaN,137301.0,R,1996-04-05,PolyGram Filmed Entertainment,Fargo,NaN
/title/tt0118589/,USA,Vondie Curtis-Hall,104,"[Drama, Music, Romance]",4273372,English,Mariah Carey,14/100,306,"18,856",2414596.0,PG-13,2001-09-21,Twentieth Century Fox Film Corporation,Glitter,2.1
/title/tt0118694/,Hong Kong,Kar Wai Wong,98,"[Drama, Romance]",2734044,Cantonese,Tony Chiu Wai Leung,85/100,345,"67,141",138340.5,PG,2001-03-09,Block 2 Pictures,In the Mood for Love,8.1
/title/tt0118926/,Spain,John Malkovich,132,"[Drama, Thriller, Crime]",2377348,English,Javier Bardem,64/100,71,"5,355",92433.0,R,2003-05-23,Fox Searchlight Pictures,The Dancer Upstairs,7.0


In [20]:
data = data[data['user_score'] >= 0]

In [21]:
len(data)

3819

> Convert meta score to a number bet 0 and 100

In [22]:
data['metacritic'] = data['metacritic'].map(lambda x: int(x.split('/')[0]))

In [23]:
# inspect highest meta score film in data

data.loc[data['metacritic'].idxmax()]

country                      USA
director       Richard Linklater
duration                     165
genres                   [Drama]
gross                   25359200
language                 English
lead              Ellar Coltrane
metacritic                   100
num_reviews                  775
num_users                223,234
opening                   387618
rating                         R
release               2014-08-15
studio           IFC Productions
title                    Boyhood
user_score                     8
Name: /title/tt1065073/, dtype: object

In [24]:
# inspect highest gross film in data

data.loc[data['gross'].idxmax()]

country                                           USA
director                                James Cameron
duration                                          162
genres                   [Action, Adventure, Fantasy]
gross                                       760507625
language                                      English
lead                                  Sam Worthington
metacritic                                         83
num_reviews                                     3,023
num_users                                     809,802
opening                                  1.276358e+07
rating                                          PG-13
release                                    2009-12-18
studio         Twentieth Century Fox Film Corporation
title                                          Avatar
user_score                                        7.9
Name: /title/tt0499549/, dtype: object

> Inspect films with meta score above 90

In [25]:
high = data[data['metacritic'] > 90]

In [26]:
high.sort(['metacritic'], ascending=False).head()

,country,director,duration,genres,gross,language,lead,metacritic,num_reviews,num_users,opening,rating,release,studio,title,user_score
/title/tt1065073/,USA,Richard Linklater,165,[Drama],25359200,English,Ellar Coltrane,100,775,"223,234",387618,R,2014-08-15,IFC Productions,Boyhood,8.0
/title/tt0457430/,Spain,Guillermo del Toro,118,"[Drama, Fantasy, War]",37623143,Spanish,Ivana Baquero,98,"1,061","419,515",501691,R,2007-01-19,Estudios Picasso,Pan's Labyrinth,8.3
/title/tt1032846/,Romania,Cristian Mungiu,113,[Drama],1185783,Romanian,Anamaria Marinca,97,170,"42,434",51712,NOT RATED,2007-09-14,Mobra Films,"4 Months, 3 Weeks and 2 Days",7.9
/title/tt2024544/,USA,Steve McQueen,134,"[Biography, Drama, History]",56667870,English,Chiwetel Ejiofor,97,643,"364,158",923715,R,2013-11-08,Regency Enterprises,12 Years a Slave,8.1
/title/tt1454468/,USA,Alfonso Cuarón,91,"[Sci-Fi, Thriller]",274086615,English,Sandra Bullock,96,"1,832","519,743",55785112,PG-13,2013-10-04,Warner Bros.,Gravity,7.9


In [27]:
data.metacritic.mean()

54.81696779261587

> Inspect films with gross below $150,000

In [28]:
low_gross = data[data['gross'] < 150000]

In [29]:
low_gross.metacritic.mean()

57.48868778280543

> Ensure that all films were released in 2000 or later

In [30]:
data['release'] = pd.to_datetime(data['release'])

In [31]:
data = data[data['release'] >= '2000-01-10 00:00:00']

> Ensure that all films are feature length (over 80 minutes)

In [32]:
data = data[data['duration'] >= 80]

In [33]:
len(data)

3780

> Add month variable

In [34]:
data['month'] = data['release'].map(lambda x: x.month)

> Clean up MPAA rating variable, consolidating `UNKNOWN`, `NOT RATED`, and `UNRATED`

In [35]:
data['rating'].value_counts()

R            1701
PG-13        1281
PG            398
UNKNOWN       142
NOT RATED     115
UNRATED        88
G              42
NC-17          13
dtype: int64

In [36]:
not_rated = data['rating'] == 'NOT RATED'
data['rating'][not_rated] = 'UNRATED'

unknown = data['rating'] == 'UNKNOWN'
data['rating'][unknown] = 'UNRATED'

data['rating'].value_counts()

/Users/emilyschuch/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/Users/emilyschuch/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


R          1701
PG-13      1281
PG          398
UNRATED     345
G            42
NC-17        13
dtype: int64

> Inspect studio variable

In [37]:
studios = data.groupby(['studio'])

studios_freq = studios.filter(lambda x: len(x) > 10)
top_studios = studios_freq.groupby(['studio']).size().order(ascending=False)[:20]


for studio in top_studios.index:
    print studio

Universal Pictures
Warner Bros.
Paramount Pictures
Twentieth Century Fox Film Corporation
New Line Cinema
Columbia Pictures
Columbia Pictures Corporation
DreamWorks SKG
Walt Disney Pictures
Touchstone Pictures
Fox Searchlight Pictures
Metro-Goldwyn-Mayer (MGM)
Screen Gems
Miramax
Fox 2000 Pictures
Dimension Films
Lionsgate
Revolution Studios
Focus Features
Summit Entertainment


> Inspect genre variable

In [38]:
all_genres = set(data['genres'].sum())

In [39]:
all_genres

{'Action',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Thriller',
 'War',
 'Western'}

> Join genre variables on underscore for unwinding as categorical variables

In [40]:
data['genres'] = data['genres'].map(lambda x: '_'.join(x))

In [41]:
data.head()

,country,director,duration,genres,gross,language,lead,metacritic,num_reviews,num_users,opening,rating,release,studio,title,user_score,month
/title/tt0035423/,USA,James Mangold,118,Comedy_Fantasy_Romance,47095453,English,Meg Ryan,44,311,"59,888",2562448.0,PG-13,2001-12-25,Konrad Pictures,Kate & Leopold,6.3,12
/title/tt0118589/,USA,Vondie Curtis-Hall,104,Drama_Music_Romance,4273372,English,Mariah Carey,14,306,"18,856",2414596.0,PG-13,2001-09-21,Twentieth Century Fox Film Corporation,Glitter,2.1,9
/title/tt0118694/,Hong Kong,Kar Wai Wong,98,Drama_Romance,2734044,Cantonese,Tony Chiu Wai Leung,85,345,"67,141",138340.5,PG,2001-03-09,Block 2 Pictures,In the Mood for Love,8.1,3
/title/tt0118926/,Spain,John Malkovich,132,Drama_Thriller_Crime,2377348,English,Javier Bardem,64,71,"5,355",92433.0,R,2003-05-23,Fox Searchlight Pictures,The Dancer Upstairs,7.0,5
/title/tt0120202/,France,David Mamet,105,Comedy_Drama,6920692,English,Philip Seymour Hoffman,75,169,"17,724",580163.0,R,2001-01-12,Filmtown Entertainment,State and Main,6.8,1


In [42]:
df = data['genres'].str.get_dummies(sep='_')
df.head(10)

,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
/title/tt0035423/,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
/title/tt0118589/,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0
/title/tt0118694/,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
/title/tt0118926/,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0
/title/tt0120202/,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
/title/tt0120667/,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
/title/tt0120679/,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
/title/tt0120737/,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
/title/tt0120753/,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0
/title/tt0120804/,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0


> Join genre categorical variables back to original dataframe and drop original genre column

In [43]:
data = data.join(df)

In [44]:
data.head()

,country,director,duration,genres,gross,language,lead,metacritic,num_reviews,num_users,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
/title/tt0035423/,USA,James Mangold,118,Comedy_Fantasy_Romance,47095453,English,Meg Ryan,44,311,"59,888",...,0,0,0,0,1,0,0,0,0,0
/title/tt0118589/,USA,Vondie Curtis-Hall,104,Drama_Music_Romance,4273372,English,Mariah Carey,14,306,"18,856",...,0,1,0,0,1,0,0,0,0,0
/title/tt0118694/,Hong Kong,Kar Wai Wong,98,Drama_Romance,2734044,Cantonese,Tony Chiu Wai Leung,85,345,"67,141",...,0,0,0,0,1,0,0,0,0,0
/title/tt0118926/,Spain,John Malkovich,132,Drama_Thriller_Crime,2377348,English,Javier Bardem,64,71,"5,355",...,0,0,0,0,0,0,0,1,0,0
/title/tt0120202/,France,David Mamet,105,Comedy_Drama,6920692,English,Philip Seymour Hoffman,75,169,"17,724",...,0,0,0,0,0,0,0,0,0,0


In [45]:
data = data.drop('genres', axis=1)

> Clean up and convert string numerical variables to integers

In [46]:
data['num_users'] = data['num_users'].map(lambda x: int(x.replace(',', '')))

data['num_reviews'] = data['num_reviews'].map(lambda x: int(x.replace(',', '')))

> Save out the cleaned dataframe

In [47]:
with open('movie_data_clean.pkl', 'w') as picklefile:
    pickle.dump(data, picklefile)

In [48]:
data.shape

(3780, 36)